In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')
nltk.download('names')
from nltk.tokenize import word_tokenize
import re
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk import pos_tag
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.corpus import names
import csv
import random

In [6]:
import transformers
from transformers import AutoModel, BertTokenizerFast

In [7]:
import torch
from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available
from transformers import BertTokenizerFast, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
import numpy as np
import random
from sklearn.model_selection import train_test_split

In [ ]:
!pip install transformers torch

In [8]:
import torch
import torch.nn as nn

In [127]:
def nltk2wn_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:                    
        return None
def lemmatize_sentence(sentence):
    lem = WordNetLemmatizer()
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))    
    wn_tagged = map(lambda x: (x[0], nltk2wn_tag(x[1])), nltk_tagged)
    res_words = []
    for word, tag in wn_tagged:
        if tag is None:                        
            res_words.append(word)
        else:
            res_words.append(lem.lemmatize(word, tag))
    return " ".join(res_words)

def preprocess_tweet(tweet):
    tweet = tweet.lower()
    tweet = re.sub(r'http\S+','',tweet)        # remove urls
    tweet = re.sub('[^a-z]',' ',tweet)
    tweet = lemmatize_sentence(tweet)

    tweet = word_tokenize(tweet)
    #add_stop = ['religion','religious']

    #tweet = [ps.stem(word) for word in tweet if not word in set(stopwords.words('english'))]
    tweet = [word for word in tweet if not word in set(stopwords.words('english'))]
    tweet = [word for word in tweet if not word in set(names.words())]

    tweet = " ".join(tweet)
    return tweet

### **Mount drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [11]:
dataset = pd.read_pickle('drive/MyDrive/Bert model/dataset.pickle')
dataset.sample(3)

,content,topic
32256,@therealzanetta FULLY explains why I wholehear...,education
84543,@Originalmugwug @Artsymarxist @fireyphoenix111...,religion
3078,James Corden teases Jennifer Lopez's appearanc...,celebrity


In [13]:
dataset.topic.value_counts()

celebrity    17724
education    17724
politics     17724
religion     17724
sports       17703
health       17158
Name: topic, dtype: int64

In [15]:
dataset.shape

(105757, 2)

### Sampling 1800sample/class = 10800
##### train=8640 , val=1080 , test=1080

In [43]:
df = dataset.groupby('topic', group_keys=False).apply(lambda x: x.sample(min(len(x), 1800)))

In [44]:
df.topic.value_counts()

celebrity    1800
education    1800
health       1800
politics     1800
religion     1800
sports       1800
Name: topic, dtype: int64

In [45]:
from sklearn.model_selection import train_test_split
x_train,x_test_val,y_train,y_test_val = train_test_split(df.content,df.topic,test_size=0.2,stratify=df.topic)

In [46]:
x_val,x_test,y_val,y_test = train_test_split(x_test_val,y_test_val,test_size=0.5,stratify=y_test_val)

In [47]:
train_df = pd.DataFrame(np.column_stack([x_train,y_train]),columns=['content','topic'])

In [48]:
test_df = pd.DataFrame(np.column_stack([x_test,y_test]),columns=['content','topic'])

In [49]:
val_df = pd.DataFrame(np.column_stack([x_val,y_val]),columns=['content','topic'])

In [138]:
train_df.content = np.vectorize(preprocess_tweet)(train_df.content)
val_df.content = np.vectorize(preprocess_tweet)(val_df.content)
test_df.content = np.vectorize(preprocess_tweet)(test_df.content)

### Load Tokenizer

In [ ]:
model_name = "bert-base-uncased"
# max sequence length for each document/sentence sample
max_length = 512

# load the tokenizer
tokenizer = BertTokenizerFast.from_pretrained(model_name, do_lower_case=True)


# tokenize the dataset, truncate when passed `max_length`, 
# and pad with 0's when less than `max_length`
train_encodings = tokenizer(train_df.content.to_list(), truncation=True, padding=True, max_length=max_length)
valid_encodings = tokenizer(val_df.content.to_list(), truncation=True, padding=True, max_length=max_length)
test_encodings = tokenizer(test_df.content.to_list(), truncation=True, padding=True, max_length=max_length)

## Encode Labels

In [140]:
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()

train_labels =  lb.fit_transform(train_df.topic)
test_labels =  lb.transform(test_df.topic)
val_labels =  lb.transform(val_df.topic)

In [141]:
label_names = pd.DataFrame({'encode':train_labels,'true':train_df.topic})

In [142]:
target_names = {}
for i,k in zip(label_names.encode,label_names.true):
  target_names[i] = k

In [143]:
target_names

{0: 'celebrity',
 1: 'education',
 2: 'health',
 3: 'politics',
 4: 'religion',
 5: 'sports'}

## converting to torch Tensors

In [144]:
class NewsGroupsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)

# convert our tokenized data into a torch Dataset
train_dataset = NewsGroupsDataset(train_encodings, train_labels)
valid_dataset = NewsGroupsDataset(valid_encodings, val_labels)
test_dataset = NewsGroupsDataset(test_encodings, test_labels)

## Load Bert Model

In [ ]:
# load the model and pass to CUDA
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=len(target_names)).to("cuda")

In [146]:
from sklearn.metrics import accuracy_score

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  # calculate accuracy using sklearn's function
  acc = accuracy_score(labels, preds)
  return {
      'accuracy': acc,
  }

In [147]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=20,   # batch size for evaluation
    warmup_steps=3000,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    load_best_model_at_end=True,     # load the best model when finished training (default metric is loss)
    logging_steps=400,               # log & save weights each logging_steps
    save_steps=400,
    evaluation_strategy="steps",     # evaluate each `logging_steps`
)

using `logging_steps` to initialize `eval_steps` to 400
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [148]:
trainer = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=valid_dataset,          # evaluation dataset
    compute_metrics=compute_metrics,     # the callback that computes metrics of interest
)

In [149]:
# train the model
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 8640
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 3240


Step,Training Loss,Validation Loss,Accuracy
400,1.385000,0.413007,0.953704
800,0.174200,0.137321,0.974074
1200,0.128700,0.138556,0.975000
1600,0.099700,0.158591,0.968519
2000,0.096200,0.142631,0.970370
2400,0.104600,0.117344,0.978704
2800,0.079400,0.117540,0.979630
3200,0.080500,0.145348,0.978704


***** Running Evaluation *****
  Num examples = 1080
  Batch size = 20
Saving model checkpoint to ./results/checkpoint-400
Configuration saved in ./results/checkpoint-400/config.json
Model weights saved in ./results/checkpoint-400/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1080
  Batch size = 20
Saving model checkpoint to ./results/checkpoint-800
Configuration saved in ./results/checkpoint-800/config.json
Model weights saved in ./results/checkpoint-800/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1080
  Batch size = 20
Saving model checkpoint to ./results/checkpoint-1200
Configuration saved in ./results/checkpoint-1200/config.json
Model weights saved in ./results/checkpoint-1200/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1080
  Batch size = 20
Saving model checkpoint to ./results/checkpoint-1600
Configuration saved in ./results/checkpoint-1600/config.json
Model weights saved in ./results/checkpoint-1600/pytorch_model.bi

TrainOutput(global_step=3240, training_loss=0.2654176871533747, metrics={'train_runtime': 1219.9114, 'train_samples_per_second': 21.247, 'train_steps_per_second': 2.656, 'total_flos': 2903863671452160.0, 'train_loss': 0.2654176871533747, 'epoch': 3.0})

## Evaluation

In [151]:
trainer.evaluate(test_dataset)

***** Running Evaluation *****
  Num examples = 1080
  Batch size = 20


{'epoch': 3.0,
 'eval_accuracy': 0.9842592592592593,
 'eval_loss': 0.08592195063829422,
 'eval_runtime': 14.1268,
 'eval_samples_per_second': 76.451,
 'eval_steps_per_second': 3.823}

## saving the fine tuned model & tokenizer

In [ ]:
model_path = "drive/MyDrive/Bert model/bert_model/"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

## Prediction

In [166]:
def get_prediction(text):
    # prepare our text into tokenized sequence
    inputs = tokenizer(text, padding=True, truncation=True, max_length=max_length, return_tensors="pt").to("cuda")
    # perform inference to our model
    outputs = model(**inputs)
    # get output probabilities by doing softmaxs
    probs = outputs[0].softmax(1)
    # executing argmax function to get the candidate label
    return target_names[probs.argmax().item()]

In [161]:
# Example #1
text = """ johnny depp and his wife amber herd at the court """
text = preprocess_tweet(text)
print(get_prediction(text))

celebrity


## Loading and testing

In [ ]:
tk = BertTokenizerFast.from_pretrained('drive/MyDrive/Bert model/bert_model/',local_files_only=True)
bert = BertForSequenceClassification.from_pretrained('drive/MyDrive/Bert model/bert_model/pytorch_model.bin',config='drive/MyDrive/Bert model/bert_model/config.json', local_files_only=True)

In [163]:
def get_prediction(text):
    # prepare our text into tokenized sequence
    inputs = tk(text, padding=True, truncation=True, max_length=max_length, return_tensors="pt")
    # perform inference to our model
    outputs = bert(**inputs)
    # get output probabilities by doing softmaxs
    probs = outputs[0].softmax(1)
    # executing argmax function to get the candidate label
    return target_names[probs.argmax().item()]

In [167]:
# Example #1
text = """ johnny deep and his ex wife are debating at the court """
text = preprocess_tweet(text)
print(get_prediction(text))

politics


## Save target_names

In [165]:
import pickle

In [108]:
with open('drive/MyDrive/Bert model/target_names.pickle','wb') as f:
  pickle.dump(target_names,f,protocol=pickle.HIGHEST_PROTOCOL)